# Utility methods to get stock prices

In [141]:
!pip install bs4 pandas fastcore seaborn sqlalchemy

import datetime
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import urllib3
urllib3.disable_warnings() # disable ssl verification warnings
from fastcore.parallel import parallel
from functools import reduce

import sqlite3
from sqlalchemy import create_engine


     |████████████████████████████████| 1.3 MB 21.6 MB/s eta 0:00:01


In [31]:
def get_todays_quotes(symbol):
    return pd.read_csv(f'https://stooq.com/q/l/?s={symbol}&f=sd2t2ohlcv&h&e=csv')

def get_last_quote(symbol):
    return get_todays_quotes(symbol)[['Date', 'Time', 'Close', 'Volume']].rename(columns={'Close':'Price'})

In [174]:
def get_stock_prices_from(url):
    def _get_stock_prices_from(url, page=1):
        page_appended_url = f"{url}&l={page}"
        res = requests.get(page_appended_url)
        
        soup = BeautifulSoup(res.text)

        quotes_table = soup.find('table', {'id': 'fth1'})
        columns = [th.text.lower() for th in quotes_table.thead.tr]
        columns

        df = pd.DataFrame()
        records = []
        for tr in quotes_table.tbody:
            records.append([
                td.text for i, td in enumerate(tr.children)
            ])

        if len(records) > 0:
            _, next_page_records = _get_stock_prices_from(url, page=page+1)
            records += next_page_records

        return columns, records
    
    
    columns, records = _get_stock_prices_from(url)
    
    return pd.DataFrame(records, columns=columns)

In [175]:
def dates_since(start_date):
    d = start_date
    while d < datetime.date.today():
        yield d
        d += datetime.timedelta(days=1)

In [185]:
def get_historical_stock_prices_from(url, date_since):
    df = pd.DataFrame()
    
    for d in dates_since(date_since):
        dated_url = f"{url}&d={d.strftime('%Y%m%d')}"
        print(dated_url)
        _df = get_stock_prices_from(dated_url)
        _df['Date'] = d
        df = df.append(_df)
        
    return df

In [187]:
df = get_historical_stock_prices_from('https://stooq.com/t/?i=523&v=1', datetime.datetime.strptime('2021-01-01', '%Y-%m-%d').date())

https://stooq.com/t/?i=523&v=1&d=20210101
https://stooq.com/t/?i=523&v=1&d=20210102
https://stooq.com/t/?i=523&v=1&d=20210103
https://stooq.com/t/?i=523&v=1&d=20210104
https://stooq.com/t/?i=523&v=1&d=20210105
https://stooq.com/t/?i=523&v=1&d=20210106
https://stooq.com/t/?i=523&v=1&d=20210107
https://stooq.com/t/?i=523&v=1&d=20210108
https://stooq.com/t/?i=523&v=1&d=20210109
https://stooq.com/t/?i=523&v=1&d=20210110
https://stooq.com/t/?i=523&v=1&d=20210111


In [162]:
engine = create_engine("sqlite:///stocks.sqlite")

In [163]:
urls_to_fetch = [
    {'url':'https://stooq.com/t/?i=523&v=1', 'name':"GPW"},
    {'url':'https://stooq.com/t/?i=534&v=1', 'name':"Crypto"}
]

df = pd.DataFrame()

for conf in urls_to_fetch:
    _df = get_stock_symbols_from(conf['url'])
    _df['Market'] = conf['name']
    df = df.append(_df)

df.to_sql('stocks', engine, if_exists='append')

ValueError: Empty table or column name specified

In [191]:
df

,symbol,name,open,high,low,last,change,volume,turnover,date,,Date
0,01C,01CYBATON,,,,,,,,,,2021-01-06
1,06N,06MAGNA,,,,,,,,,,2021-01-06
2,08N,08OCTAVA,,,,,,,,,,2021-01-06
3,11B,11BIT,,,,,,,,,,2021-01-06
4,1AT,ATAL,,,,,,,,,,2021-01-06
...,...,...,...,...,...,...,...,...,...,...,...,...
844,ZRE,ZREMB,0.765,0.775,0.735,0.770,-0.65%,17.7k,13.5k,Jan 11,,2021-01-11
845,ZST,ZASTAL,31.8,34.2,31.4,33.6,+3.07%,6.51k,157k,Jan 11,,2021-01-11
846,ZUE,ZUE,4.20,4.28,4.12,4.28,0.00%,2.27k,9.49k,Jan 11,,2021-01-11
847,ZUK,STAPORKOW,2.92,2.98,2.80,2.80,0.00%,25.1k,51.9k,Jan 11,,2021-01-11
